In [1]:
import os

# Modules to read, process and output events.
from core.dsl.transformer.mp4_to_sliding_events import Mp4ToSlidingEvents
from core.dsl.transformer.mp4_to_stacking_events import Mp4ToStackingEvents
from core.dsl.transformer.mp4_to_singular_events import Mp4ToSingularEvents
from core.dsl.transformer.frames_by_event_batches import EventBatchToFrames
from core.dsl.sink.event_writer import EventWriter
from core.dsl.transformer.frames_by_timestamps import FramesByTimestamps
from core.dsl.transformer.event_to_intensity_predictor import AsymptoticIntensityPredictor
from core.dsl.transformer.batch_throughput_limiter import BSync
from core.dsl.source.mp4_reader import Mp4Reader
from core.dsl.sink.window import Window
from core.dsl.source.events_reader import EventReader
from core.dsl.sink.mp4_writer import Mp4Writer
from core.dsl.transformer.mp4_to_greyscale import MP4ToGreyscale

In [2]:
import metavision_core.utils.samples as samples
samples.get_sample('driving_sample.raw', 'samples/events/')

Metavision SDK utilizes raw/dat files to read events. Let's begin by downloading samples from their website.

In [3]:
def load_event_samples(directory, filetypes):
    absolute_path = os.path.abspath(directory)
    files = os.listdir(directory)
    return {file[:-4]: f'{absolute_path}/{file}' for file in files if file[-4:] in filetypes}

# Authentic events from event cameras.
event = load_event_samples('samples/events/', ['.raw', '.dat'])

# Conventional frame based video not derived from event cameras.
mp4 = load_event_samples('samples/mp4/', ['.mp4'])

# Conventional video deconstructed to synthetic events.
decon = load_event_samples('samples/decon/', ['.raw', '.dat'])

# Events reconstructed to regular video.
recon = load_event_samples('samples/recon/', ['.mp4'])

Let's start with something simple, like streaming a regular video.

In [4]:
# This is the module that will feed the video stream from the filesystem.
mp4_in = Mp4Reader(mp4['formula1'])

# This window will consume whatever stream of images you provide it, and display it in a separate window.
window = Window('formula1 mp4-stream')

# Now we combine the input and output module with >> operator.
# Note that both modules must have the same transfer datatype to be compatible, i.e (mp4 -> mp4) or (event -> event).
# If the datatype were different, such as (event -> mp4) or (mp4 -> event), converters would have to be placed as well.
mp4_in >> window

Video-stream were probably going a bit fast, right? Let's try to synchronize it with the system-clock.

In [5]:
# The modules are mutable. Never reuse old instances.
mp4_in = Mp4Reader(mp4['formula1'])
window = Window('Synced formula1 mp4-stream')

# This module treat events and frames equal. It simply restricts the number of invocations per second.
# You can use this module for both event and frame datatypes. Note that this module is blocking.
frame_sync = BSync(batch_per_second=24.0)

mp4_in >> frame_sync >> window

We can also compute the greyscale version in real-time.
This will be important during this thesis in order to have a comparable ground-truth to reconstructed data.
Only greyscale information will be reconstructed, so there's no sense comparing it with rgb video.

In [12]:
mp4_in = Mp4Reader(mp4['formula1'])

# Accepts any frame, and outputs greyscale variant calculated by average method.
greyscale_gen = MP4ToGreyscale()
window = Window('Synced formula1 mp4-stream')

mp4_in >> greyscale_gen >> window

Now lets render raw events instead to better grasp the difference.

In [13]:
# We use a dedicated event reader for this purpose.
# Each individual event has a timestamp denoting when it was created.
# Delta_t denotes the timespan for each event batch.
event_in = EventReader(event['driving_sample'], delta_t=1e4)

# Simply render events from each batch to a frame. This frame generator is highly influenced by delta_t.
# Increasing it will yield more events per frame.
batch_frame_generator = EventBatchToFrames()

# No explaining needed.
window = Window('Frames from event batches.')

# Events are read, then converted to frame, lastly the frames are feed to the window.
# events -> frames -> void
event_in >> batch_frame_generator >> window

We can also synchronize events as we previously did with frames.

In [22]:
event_in = EventReader(event['back6'], delta_t=1e6)
batch_frame_generator = EventBatchToFrames()
window = Window('Synchronized frames from event batches')

# Lets use Bsync (Batch synchronizer)
frame_sync = BSync(batch_per_second=30.0)

# Events are read, then converted to frame, then synchronized, lastly the frames are feed to the window.
# events -> frames -> frames -> void
event_in >> batch_frame_generator >> frame_sync >> window

Now instead of generating frames based on batches, let's use another approach, namely the timestamps on the events itself.

In [17]:
event_in = EventReader(event['driving_sample'], delta_t=1e2)
window = Window('Raw to frames conversion')

# Not influenced by batch sizes and adjustments to delta_t. Create frames based on timestamps instead.
# Note that adjusting fps will only influence the video speed, not smoothness.
# This is intended behaviour as the algorithm correlates time with the timestamps, not real-time.
timestamp_frame_generator = FramesByTimestamps(fps=200)

event_in >> timestamp_frame_generator >> window

To account for real-time playback problem in the cell above, we incorporate a syncing module like previously.

In [16]:
event_in = EventReader(event['driving_sample'], delta_t=1e6)
window = Window('Synced event stream')
timestamp_frame_generator = FramesByTimestamps(fps=30)

# Accounting for real-time playback.
# Note: might not yield correct result due to lagging.
frame_sync = BSync(batch_per_second=30.0)

event_in >> timestamp_frame_generator >> frame_sync >> window

Now lets try to reconstruct event data in real-time. This algorithm counts events consecutively and based on the individual
events' polarity, either increase or decrease the greyscale value of its respective pixel position. Due to information loss in event data,
artifacts such as ghosting will occur. This model uses gaussian filters and decaying factors to compensate for that. You may need to adjust the individual parameters through trial and error to find an optimal setting.

In [6]:
event_in = EventReader(event['driving_sample'], delta_t=1e4)

reconstructor = AsymptoticIntensityPredictor(
    gaussian_filter_sigma=0.3,
    intensity_decay=0.2,
    intensity_impedance=1.0
)

window = Window('Reconstruction')

event_in >> reconstructor >> window

Only Authentic events have been visualized so far. Problem with reconstructing authentic event data is that we possess no
actual ground-truth (correct data correspondence). Luckily, it's way easier to mimic events from regular pictures than vice-versa.
We do this by computing pixel intensity gradients from frame to frame. By doing this, we possess something close to actual real data, but most importantly
the actual solution to what a proper reconstruction should look like. Some key differences to keep in mind:

* Synthetic events typically have lower temporal resolution.
* Lower dynamic range.
* Less temporal noise.

But having actual intensity information can be argued to outweigh these drawbacks for testing purposes.

In the following cell, we will deconstruct an authentic conventional rgb video to 'singular' events. 'singular' is not an acknowledged technical term
used to describe events, but we introduced it in this thesis to differentiate it from other deconstruction algorithms that we investigated in this thesis. This deconstruction algorithm is probably the best approximation to how actual events work, while the other 2 are fictionary used to investigate better ways to generate events which may lead to less intensity loss. How 'singular events are generated:

1. Using pixel intensity information, count how many threshold has been bypassed.
2. Compare it to the count of the previous frame. Regardless of intensity difference between the 2 frames, fire only 1 event. If intensity changed by 2 or more
   Only 1 event shall be fired. This 'fire 1 event scheme' is one of several reasons to intensity loss in event cameras.
3. Set the new frame as the new state.
4. Repeat

In [14]:
mp4_in = Mp4Reader(mp4['formula1'])
batch_frame_generator = EventBatchToFrames()
window = Window('Deconstruction to singular events.')

# The deconstructor producing singular events. You may adjust threshold manually here.
# This is not possible with most actual event cameras.
deconstructor = Mp4ToSingularEvents(threshold=0.1)

# Remember that deconstructor outputs event data.
# You must generate frames from them before sending it to the window.
# frames -> events -> frames -> void
mp4_in >> deconstructor >> batch_frame_generator >> window

In the following cell, we demonstrate deconstruction as well, but now we're using 'sliding' events instead.
This algorithm woks very similar to the previous one, but have some subtle difference in threshold placement. Note that this algorithm doesn't render events
like the prophesee event camera does, but do yield similar result from visual assessment. One benefit of 'sliding' events contra 'singular' events is that compression artefacts from regular videos is less visible. How it works:

1. Take a frame and calculate greyscale intensity information. Note no floor-division here.
2. Find out which pixel exceeds that pixels current +- a threshold revolving around it. Those that do are an event.
3. Register pixel with events as new states, otherwise leave their states as it is.

In contrast to singular events algorithms, pixel states are not discrete values.

In [15]:
mp4_in = Mp4Reader(mp4['formula1'])
batch_frame_generator = EventBatchToFrames()
window = Window('Deconstruction to sliding events.')

# The deconstructor producing sliding events. You may adjust threshold manually here.
# This is not possible with most actual event cameras.
deconstructor = Mp4ToSlidingEvents(threshold=0.1)

# Remember that deconstructor outputs event data.
# You must generate frames from them before sending it to the window.
# frames -> events -> frames -> void
mp4_in >> deconstructor >> batch_frame_generator >> window

Lastly, we demonstrate 'stacking' events. This event scheme aimed to solve the event-to-intensity problem due to only firing 1 event when the intensity difference
where many times higher. This algorithm works exactly like the singular events algorithm, but compensated for larger changes by firing more events. How it works:

1. Using pixel intensity information, count how many threshold has been bypassed.
2. Compare it to the count of the previous frame. Judging by the discrete difference, fire exactly that many events.
3. Set the new frame as the new state.
4. Repeat.

NB! This algorithm requires much more memory due to larger event yield. If you choose to record the output, be sure to watch the disk usage frequently
as the files sizes will be significant.

In [ ]:
mp4_in = Mp4Reader(mp4['formula1'])
batch_frame_generator = EventBatchToFrames()
window = Window('Deconstruction to stacking events.')

# The deconstructor producing stacking events. You may adjust threshold manually here.
# This is not possible with most actual event cameras.
deconstructor = Mp4ToStackingEvents(threshold=0.07)

# Remember that deconstructor outputs event data.
# You must generate frames from them before sending it to the window.
# frames -> events -> frames -> void
mp4_in >> deconstructor >> batch_frame_generator >> window

Here we demonstrate mp4 roundtrip reconstruction, i.e. deconstruction of mp4 and reconstruct it back again.

In [9]:
mp4_in = Mp4Reader(mp4['arnold-arm-wrestle'])

deconstructor = Mp4ToSlidingEvents(threshold=0.01)

reconstructor = AsymptoticIntensityPredictor(
    gaussian_filter_sigma=0.5,
    intensity_decay=0.05,
    intensity_impedance=1.0,
)

window = Window('Mp4 roundtrip integrity test')


mp4_in >> deconstructor >> reconstructor >> window

Here we demonstrate event roundtrip deconstruction, i.e. reconstruction of events, and then deconstruction back to events.

In [6]:
event_in = EventReader(event['driving_sample'])

reconstructor = AsymptoticIntensityPredictor(
    gaussian_filter_sigma=0.7,
    intensity_decay=0.0,
    intensity_impedance=1.0,
)

deconstructor = Mp4ToSlidingEvents(threshold=0.01, fps=30)

batch_frame_generator = EventBatchToFrames()

window = Window('Mp4 roundtrip integrity test')


event_in >> reconstructor >> deconstructor >> batch_frame_generator >> window

Event deconstruction can be slow and tedious. Use this command line tool record a sample for multiple use.
Feel free to cancel the process anytime to preview the recording. Dat files won't be corrupted by this.

In [17]:
mp4_input_path = mp4['formula1']
dat_output_path = './samples/decon/my-formula1-decon.dat'
threshold = 0.01
event_mode = 'stacking'

%run mp4_to_dat.py -i $mp4_input_path -o $dat_output_path -et $threshold -em $event_mode

KeyboardInterrupt: 